# Feature Engineering: ICU Traversal

## Features Extracted
### Explanatory Variables (`X`)

  * **`n-trav`**: The number of total ward transfers in a hospital admission.
  * **`n_icutrav`**: The number of ICU-specific ward transfers in a hospital admission.
  * **`n_readm`**: The total number of ICU readmissions for the patient over the course of the study. (*not specific to hadm_id)
  * **`n_icustays`**: The number of unique ICU stays for a hospital admission.
  * Transfer Pairs
    * **`nonicu-MICU`**	
    * **`nonicu-SICU`**	
    * **`nonicu-TSICU`**	
    * **`nonicu-CSRU`**	
    * **`MICU-MICU`**	
    * **`TSICU-TSICU`**	
    * **`nonicu-CCU`**	
    * **`CCU-CCU`**
    * **`CSRU-CSRU`**	
    * **`SICU-SICU`**	
 
### Dependent Variables/Response (`Y`) 
  * **`avg_iculos`**: The average ICU length of stay.

In [33]:
import psycopg2
from psycopg2 import extras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

%matplotlib inline

In [34]:
def cursor_connect(cursor_factory=None):
    """
    Connects to the DB and returns the connection and cursor, ready to use.
    
    Parameters
    ----------
    cursor_factory : psycopg2.extras
    
    Returns
    -------
    (psycopg2.extensions.connection, psycopg2.extensions.cursor)
        A tuple of (psycopg2 connection, psycopg2 cursor).
    """
    #DB connection
    conn = psycopg2.connect(dbname="mimic", user="mimic", host="localhost", port="2345",
                            password="oNuemmLeix9Yex7W")
    if not cursor_factory:
        cur = conn.cursor()
    else:
        cur = conn.cursor(cursor_factory=cursor_factory)
    return conn, cur

def exec_query(query, curs_dict=True):
    """
    Execute query and returns the SQL output.
    
    Parameters
    ----------
    query: string containing SQL SELECT command
    curs_dict: dict cursor factory (output is dict)
    
    Returns
    -------
    rows: list of rows/records (SQL output)
    """
    if curs_dict == True: 
        conn, cur = cursor_connect(psycopg2.extras.DictCursor)
    else:
        conn, cur = cursor_connect()
    cur.execute(query)
    rows = cur.fetchall()
    return rows

## N<sub>readmissions</sub>
Feature: The number of ICU readmissions a patient has had. The value is not tied to a hospital admission and is the same for the same `subjectid`, despite multiple records/rows.

In [35]:
# query
q_icupat="""SELECT * FROM
    (SELECT subject_id, COUNT(icustay_id) AS n_icustays
    FROM icustays
    GROUP BY subject_id) AS sub_q
WHERE n_icustays > 1;"""

# Query output
icupat = exec_query(q_icupat, False)
df_icupat = pd.DataFrame(icupat, columns=['subjectid', 'n_icustays'])
df_icupat

n_readm = pd.Series(df_icupat.n_icustays - 1, name='n_readm')
df_icu = pd.concat([df_icupat.subjectid, n_readm], axis=1)
df_icu.head()

,subjectid,n_readm
0,7,1
1,17,1
2,21,1
3,23,1
4,34,1


## N<sub>trav</sub>
Feature: The number of total ward transfers for each patient's hospital admission.

In [36]:
q_mult= """SELECT subject_id, hadm_id, icustay_id, eventtype,
prev_careunit, curr_careunit, prev_wardid, curr_wardid, intime, outtime, los
FROM transfers;"""
mult_trav = exec_query(q_mult, False)
mult_col = ['subjectid', 'hadmid', 'icustayid', 'eventtype', 'prev_cu', 'curr_cu', 
       'prev_wardid', 'curr_wardid', 'intime', 'outtime', 'los']
df_mult = pd.DataFrame(mult_trav, columns = mult_col)
df_mult.replace(to_replace='', value=np.nan, inplace=True, regex=True)

# filter for ICU patients with readmissions
filter_preadm = list(df_icupat.subjectid)
df_mult_readm = df_mult[df_mult.subjectid.isin(filter_preadm)]

# filter for exclusion of neonate patients
df_mult_readm = df_mult_readm[df_mult_readm['prev_cu'] != 'NWARD']
df_mult_readm = df_mult_readm[df_mult_readm['prev_cu'] != 'NICU']
df_mult_readm = df_mult_readm[df_mult_readm['curr_cu'] != 'NWARD']
df_mult_readm = df_mult_readm[df_mult_readm['curr_cu'] != 'NICU']

In [37]:
df_mult_readm_grp = df_mult_readm.groupby(['subjectid', 'hadmid']).size()
df_mult_readm_grp = df_mult_readm_grp.to_frame(name='n_trav').reset_index()

# join DF on subjectid to add n_icustays col
df_icu1 = pd.merge(df_mult_readm_grp, df_icu, on='subjectid', 
                        how='left')
df_icu1.head()

,subjectid,hadmid,n_trav,n_readm
0,17,161087,4,1
1,17,194023,4,1
2,21,109451,6,1
3,21,111970,2,1
4,23,124321,5,1


## N<sub>icutrav</sub>
Feature: The number of total *ICU* ward transfers for each patient's hospital admission. Only transfers into the ICU, which is assigned a `icustay_id` is accounted for.

In [38]:
df_mult_readm_icu = df_mult_readm[df_mult_readm.icustayid.notnull()==True]
df_mult_readm_hadm = df_mult_readm_icu.groupby(['subjectid', 'hadmid']).size().to_frame('n_icutrav').reset_index()

# join DF  on subjectid to add n_readm col
df_icu2 = pd.merge(df_icu1, df_mult_readm_hadm.loc[:, ['hadmid', 'n_icutrav']], 
                   on='hadmid', how='inner')
df_icu2.head()

,subjectid,hadmid,n_trav,n_readm,n_icutrav
0,17,161087,4,1,1
1,17,194023,4,1,1
2,21,109451,6,1,2
3,21,111970,2,1,1
4,23,124321,5,1,1


## N<sub>ICU Stays</sub>
Feature: the number of unique ICU stays for a patient's hospital admission.

In [39]:
# query
q_icustay = """SELECT subject_id, hadm_id, COUNT(DISTINCT icustay_id)
FROM transfers
GROUP BY subject_id, hadm_id;
"""

# Query output
icustay = exec_query(q_icustay, False)
df_icustay = pd.DataFrame(icustay, columns=['subjectid', 'hadmid', 
                                            'n_icustays'])
# join DF  on subjectid to add n_readm col
df_icu3 = pd.merge(df_icu2, df_icustay.loc[:, ['hadmid', 'n_icustays']], 
                   on='hadmid', how='inner')
df_icu3.head()

,subjectid,hadmid,n_trav,n_readm,n_icutrav,n_icustays
0,17,161087,4,1,1,1
1,17,194023,4,1,1,1
2,21,109451,6,1,2,1
3,21,111970,2,1,1,1
4,23,124321,5,1,1,1


## Traversal Pairs

In [40]:
df_trav_copy = df_mult_readm_icu.copy()
df_trav_copy.prev_cu.fillna('nonicu', inplace=True)
df_trav_copy.curr_cu.fillna('nonicu', inplace=True)
df_trav_copy['trans'] = df_trav_copy.prev_cu+'-'+df_trav_copy.curr_cu

df_toppairs = df_trav_copy.trans.value_counts(ascending=False).to_frame()
df_top = df_toppairs.transpose().iloc[:, 0:11] # transpose to columns
df_top

,nonicu-MICU,nonicu-SICU,nonicu-CCU,nonicu-CSRU,nonicu-TSICU,MICU-MICU,CSRU-CSRU,SICU-SICU,TSICU-TSICU,CCU-CCU,CCU-MICU
trans,11580,4284,3702,3458,1988,1381,1114,427,266,257,251


In [41]:
list(df_top.columns)

['nonicu-MICU',
 'nonicu-SICU',
 'nonicu-CCU',
 'nonicu-CSRU',
 'nonicu-TSICU',
 'MICU-MICU',
 'CSRU-CSRU',
 'SICU-SICU',
 'TSICU-TSICU',
 'CCU-CCU',
 'CCU-MICU']

In [42]:
from collections import Counter
# a = dict(Counter(df_trav_copy[df_trav_copy.subjectid==34].trans))
# a['subjectid']=34
# a

sid = list(df_trav_copy.subjectid.value_counts().index) # unique subject_id

main_d = dict()
for subj in sid:
    pair_d = dict(Counter(df_trav_copy[df_trav_copy.subjectid==subj].trans))
    pair_d['subjectid'] = subj # add subjectid key
    main_d[subj] = pair_d

In [43]:
df_pairct = pd.DataFrame.from_dict(main_d, orient='index')

# drop non-top trans pair cols
pairs_drop = list(df_toppairs.iloc[10:].index)
df_pairct.drop(pairs_drop, axis=1, inplace=True) 

df_icu4 = pd.merge(df_icu3, df_pairct, on='subjectid', how='left')
df_icu4.head()

,subjectid,hadmid,n_trav,n_readm,n_icutrav,n_icustays,nonicu-MICU,nonicu-SICU,nonicu-TSICU,nonicu-CSRU,MICU-MICU,TSICU-TSICU,nonicu-CCU,CCU-CCU,CSRU-CSRU,SICU-SICU
0,17,161087,4,1,1,1,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,17,194023,4,1,1,1,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,21,109451,6,1,2,1,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
3,21,111970,2,1,1,1,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
4,23,124321,5,1,1,1,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


### Probability Transformation

In [44]:
df_mult_readm_icu = df_mult_readm[df_mult_readm.icustayid.notnull()==True]
df_test = df_mult_readm_icu.copy()
df_test.prev_cu.fillna('nonicu', inplace=True)
df_test.curr_cu.fillna('nonicu', inplace=True)

pair_prob = pd.crosstab(df_test.prev_cu, 
            df_test.curr_cu)/pd.crosstab(df_test.prev_cu,
                                           df_test.curr_cu).sum()
pair_prob

curr_cu,CCU,CSRU,MICU,SICU,TSICU
prev_cu,,,,,
CCU,0.058462,0.029022,0.018337,0.009021,0.008323
CSRU,0.039126,0.230929,0.008767,0.011963,0.007491
MICU,0.039126,0.012852,0.100891,0.036478,0.032043
SICU,0.012056,0.006633,0.015488,0.083742,0.014149
TSICU,0.009099,0.003731,0.010520,0.018631,0.110695
nonicu,0.842129,0.716833,0.845996,0.840165,0.827299


In [45]:
df_pairprob = pair_prob.unstack().to_frame(name='prob').reset_index()
df_pairprob['trans'] = df_pairprob.prev_cu+'-'+df_pairprob.curr_cu
df_pairprob.drop(['curr_cu', 'prev_cu'], axis=1, inplace=True)
df_pairprob.set_index('trans', drop=True, inplace=True)
df_pairprob.sort_values('prob', ascending=False).head()

,prob
trans,
nonicu-MICU,0.845996
nonicu-CCU,0.842129
nonicu-SICU,0.840165
nonicu-TSICU,0.827299
nonicu-CSRU,0.716833


In [46]:
pairs = ['nonicu-MICU', 'nonicu-SICU', 'nonicu-TSICU', 'nonicu-CSRU',
         'MICU-MICU', 'TSICU-TSICU', 'nonicu-CCU', 'CCU-CCU', 'CSRU-CSRU',
         'SICU-SICU']
df_icu4_copy = df_icu4.copy()

for elem in pairs:
    df_icu4_copy[elem].fillna(0, inplace=True)
    df_icu4_copy[elem] = np.round(df_icu4_copy[elem] * df_pairprob.loc[elem].values[0],3)
df_icu4_copy.head()

,subjectid,hadmid,n_trav,n_readm,n_icutrav,n_icustays,nonicu-MICU,nonicu-SICU,nonicu-TSICU,nonicu-CSRU,MICU-MICU,TSICU-TSICU,nonicu-CCU,CCU-CCU,CSRU-CSRU,SICU-SICU
0,17,161087,4,1,1,1,0.000,0.00,0.0,1.434,0.0,0.0,0.000,0.000,0.0,0.0
1,17,194023,4,1,1,1,0.000,0.00,0.0,1.434,0.0,0.0,0.000,0.000,0.0,0.0
2,21,109451,6,1,2,1,0.846,0.00,0.0,0.000,0.0,0.0,0.842,0.058,0.0,0.0
3,21,111970,2,1,1,1,0.846,0.00,0.0,0.000,0.0,0.0,0.842,0.058,0.0,0.0
4,23,124321,5,1,1,1,0.000,0.84,0.0,0.717,0.0,0.0,0.000,0.000,0.0,0.0


# Response Variable

## Average ICU LOS
Feature (Response): The average length of stay in the ICU (`icustay_id`) for each patient's hospital admission.
  * Feature is the response/dependent variable (`y`)

**Feature transformation** The probability of the feature combination pair is used as weight and applied onto the count for each row.

**Note**: The overall hospital admission LOS may be extracted by taking the aggregate mean of the DataFrame, after it has been grouped by `subject_id`.

In [47]:
# Average ICU LOS
avgiculos = df_mult_readm_icu.groupby(['subjectid', 'hadmid'])['los'].mean()
df_avgiculos = avgiculos.to_frame(name='avg_iculos').reset_index()

# Merge
df_icu5 = pd.merge(df_icu4_copy, df_avgiculos.loc[:,['hadmid', 'avg_iculos']], on='hadmid', how='left')
# df_icu5.groupby(['subjectid'])['avg_iculos'].mean() # overall LOS
df_icu5.head()

,subjectid,hadmid,n_trav,n_readm,n_icutrav,n_icustays,nonicu-MICU,nonicu-SICU,nonicu-TSICU,nonicu-CSRU,MICU-MICU,TSICU-TSICU,nonicu-CCU,CCU-CCU,CSRU-CSRU,SICU-SICU,avg_iculos
0,17,161087,4,1,1,1,0.000,0.00,0.0,1.434,0.0,0.0,0.000,0.000,0.0,0.0,21.110
1,17,194023,4,1,1,1,0.000,0.00,0.0,1.434,0.0,0.0,0.000,0.000,0.0,0.0,49.700
2,21,109451,6,1,2,1,0.846,0.00,0.0,0.000,0.0,0.0,0.842,0.058,0.0,0.0,70.825
3,21,111970,2,1,1,1,0.846,0.00,0.0,0.000,0.0,0.0,0.842,0.058,0.0,0.0,200.750
4,23,124321,5,1,1,1,0.000,0.84,0.0,0.717,0.0,0.0,0.000,0.000,0.0,0.0,28.470


In [16]:
print df_icu5.shape
df_icu5.describe()

(19719, 17)


,subjectid,hadmid,n_trav,n_readm,n_icutrav,n_icustays,nonicu-MICU,nonicu-SICU,nonicu-TSICU,nonicu-CSRU,MICU-MICU,TSICU-TSICU,nonicu-CCU,CCU-CCU,CSRU-CSRU,SICU-SICU,avg_iculos
count,19719.000000,19719.000000,19719.000000,19719.000000,19719.000000,19719.000000,19719.000000,19719.000000,19719.000000,19719.000000,19719.000000,19719.000000,19719.000000,19719.000000,19719.000000,19719.000000,19718.000000
mean,32967.844211,149877.695877,4.811907,2.854607,1.542167,1.184492,1.931784,0.520533,0.203406,0.290207,0.028672,0.003745,0.490770,0.002325,0.030563,0.005142,83.350849
std,27767.554089,28882.239436,2.243329,4.024724,0.931596,0.460578,3.148073,0.943739,0.474340,0.574893,0.081099,0.022971,0.876495,0.012589,0.110448,0.024375,111.151381
min,17.000000,100001.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000
25%,11700.000000,125021.500000,3.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27.570000
50%,23707.000000,149836.000000,4.000000,2.000000,1.000000,1.000000,0.846000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,49.200000
75%,52114.000000,174547.000000,6.000000,3.000000,2.000000,1.000000,2.538000,0.840000,0.000000,0.717000,0.000000,0.000000,0.842000,0.000000,0.000000,0.000000,92.097500
max,99982.000000,199995.000000,25.000000,40.000000,11.000000,7.000000,28.764000,10.082000,4.136000,6.451000,1.009000,0.443000,9.263000,0.175000,1.847000,0.335000,2373.310000
